<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/prep_data/download_surf_GLORYS12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Purpose is to Download
-  Hurricane data (track, surface pressure, wind) from [tropycal](https://tropycal.github.io/tropycal/install.html).
- [Ocean-reanalysis data from GLorys12](https://data.marine.copernicus.eu/product/GLOBAL_ANALYSISFORECAST_PHY_001_024/services) will be gathered from the [CMEMS](https://data.marine.copernicus.eu/products); dataset specific info (may change with dataset, you need to fill it in carefully by looking up CMEMS).


  **Needed inputs**:
  1. basin (e.g., `north_atlantic`)
  2. year (e.g., `2024`)
  4. time_delta (e.g., `5`)
  5. dLon, dLat (e.g., `5, 5`)
  6. List of _priority_ storms, see below.

### Install modules

In [ ]:
!pip install tropycal
!pip install cartopy
!pip install copernicusmarine

In [ ]:
import copernicusmarine
from tropycal import tracks
import xarray as xr
import numpy as np
import pandas as pd

In [ ]:
def get_cmems_data(dsetID, vNames, lon_start, lon_end, lat_start, lat_end, time_start, time_end):

  data_request = {"dataset_id" : dsetID,
    "longitude" : [lon_start, lon_end],
    "latitude" : [lat_start, lat_end],
    "time" : [time_start, time_end],
    "variables" : vNames}

  cms_data =copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"])

  return cms_data

### User Inputs

In [ ]:
myBasin = 'north_atlantic' # basin
year = 2024 # year
time_delta = 5 # days before/after storm
dLon, dLat = [5, 5] # plot extra data outside track bounds (in degrees)

# List of _priority_ storms.
# Added a few extra ones (open ocean). Kirk (2024); Franklin (2023), Lee (2023)

if year == 2024:
  pr_st = ["oscar", "milton", "helene", "francine", "ernesto", "debby", "beryl", "kirk"]
elif year == 2023:
  pr_st = ["philippe", "ophelia", "nigel", "idalia", "gert p2", "gert p1", "don p2", "don p1", "bret", "franklin", "lee"]
elif year == 2022:
  pr_st = ["nicole", "ian", "gaston", "fiona", "earl", "danielle"]
else:
  print("List of priority storms not yet coded for year:\t {}"%format(year))

In [ ]:
# Path to save data:
dPath = f'data/{myBasin}/{year}/'
!mkdir -p $dPath
print(dPath)

### Download hurricane track data (entire chosen `year`)

In [ ]:
# Initialize
# https://tropycal.github.io/tropycal/api/generated/tropycal.tracks.TrackDataset.html#tropycal.tracks.TrackDataset

basin = tracks.TrackDataset(basin=myBasin, source='hurdat', include_btk=True)
season = basin.get_season(year)
print(f'Downloading data for...{season}')

# save data
season_data=season.to_dataframe()
fName = dPath + 'hurdat2_{}_{}.csv'.format(myBasin, year)
season_data.to_csv(fName)
print(f'Saved to hurricane track data:\t{fName}')

In [ ]:
# Set Copernicus credentials
CMEMS_username, CMEMS_passwd = ["sakella", "HbFPyP9M"]
copernicusmarine.login(username=CMEMS_username, password=CMEMS_passwd)

In [ ]:
# CMEMS dataset IDs - Reanalysis GLORYS12
# https://catalogue.marine.copernicus.eu/documents/PUM/CMEMS-GLO-PUM-001-024.pdf
# Hourly mean surface (2d) fields: cmems_mod_glo_phy_anfc_0.083deg_PT1H-m

dset_oras = "cmems_mod_glo_phy_anfc_0.083deg_PT1H-m"
vars_gloys12 = ["zos", "so", "thetao", "uo", "vo"]
vNames = ["SSH", "SSS", "SST", "SSU", "SSV"]

### Focus on _priority_ storms (as defined above) only.

In [ ]:
hurr_names = season_data['name']
hurr_ids = season_data['id']

print("\nDownloading GLORYS12 data for the following storms:\n")
for idx, hurr_id in enumerate(hurr_ids):
    hurr_name = hurr_names.iloc[idx]
    if hurr_names.iloc[idx].lower() in pr_st:
      print("\nHurricane...\t{}, [{}]".format(hurr_name, hurr_id))
      hurr=basin.get_storm(hurr_id)
      # Hurricane formation and dissipation dates (yyyymmdd)
      print("Formed on: {},\t dissipated on: {}".
      format(hurr.time[0].strftime('%Y-%m-%d'), hurr.time[-1].strftime('%Y-%m-%d')))
      t0 = hurr.time[0] - pd.Timedelta(days=time_delta)
      t1 = hurr.time[-1] + pd.Timedelta(days=time_delta)

      storm_fName = str(year) + "_" + myBasin + "_" + hurr_name + '.nc'
      track_file = dPath + storm_fName
      hurr.to_xarray().to_netcdf(track_file)
      print("Saved track info to:\n{}".format(track_file))

      # Read back the saved data (for formatting) and use it here onward
      hurr=xr.open_dataset(track_file)

      lon_s, lon_e = [hurr.lon.values.min()-dLon, hurr.lon.values.max()+dLon]
      lat_s, lat_e = [hurr.lat.values.min()-dLat, hurr.lat.values.max()+dLat]
      t_s, t_e = [t0.strftime('%Y-%m-%d'), t1.strftime('%Y-%m-%d')]
      !mkdir -p $dPath/$hurr_name

      # download GLORYS12 datasets
      for vid, vName in enumerate(vNames):
        print("Downloading:\t{}".format(vName))
        oras_data_path = dPath +  hurr_name + '/'+'GLO12_'+ vName + '_' +str(year)+'_'+hurr_name
        var_oras = vars_gloys12[vid]
        print(var_oras, oras_data_path)
        !copernicusmarine subset --dataset-id $dset_oras --username $CMEMS_username --password $CMEMS_passwd -o $oras_data_path --variable $var_oras --minimum-latitude $lat_s --maximum-latitude $lat_e --minimum-longitude $lon_s --maximum-longitude $lon_e --start-datetime $t_s --end-datetime $t_e --service arco-geo-series
      !tar -cvf $hurr_name+'.tar' $dPath+$hurr_name+'/*'
      !rm -rf $dPath+$hurr_name+'/'

### Size of datasets > 15GB that is available on _unpaid_ account.